In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# table_data.py
import urllib3
from datetime import datetime

In [5]:
respiratory_data= pd.read_csv('respiratory_data.csv')
http = urllib3.PoolManager()

In [6]:
#NCT Numbers from the clinical trial numbers

list_NCT= list(respiratory_data['NCT Number'])
URL = 'https://clinicaltrials.gov/ct2/show/record/'

final_data= []

for NCT in list_NCT:
    
    #request permission 
    req= http.request('GET', URL + str(NCT) + '/')
    
    #extract data
    soup = BeautifulSoup(req.data, 'html.parser')
    
    #find table in webpage with the data 
    table = soup.find('table', attrs={'class': 'ct-data_table tr-data_table tr-tableStyle'})

    #find all table rows
    rows = table.find_all('tr')
    
    #print(rows)
    
    #extract the variable and value from rows in webpages
    data= []
    
    for i in range(len(rows)):
        if rows[i].th is None:
            name= 'NA'
        else:
            name= rows[i].th.text
        if rows[i].td is None:
            value= 'NA'
        else:
            value= rows[i].td.text
        data.append(tuple([name, value]))
    
    #create dataframe with all the values
    df = pd.DataFrame(data, columns=['Name', 'Value'])
    #print(df)
    
    #extract enrollment, NCT number, and Countries 
    my_df= df[df['Name'].str.contains('Enrollment') | df['Name'].str.contains('NCT') | df['Name'].str.contains('Listed Location Countries') ].reset_index()
    
    #get rid of unnecessary strings in the values 
    my_df['Value']= my_df['Value'].str.rstrip()
    
    my_df['Value'][2].replace("\xa0", "")
    #print(my_df)
    
    #print(values_df)
    #append data we want to our final list
    final_data.append(list(my_df['Value']))

In [7]:
#Create data frame with our final list 
enrollment_countries_respiratory = pd.DataFrame(final_data, columns=['Actual', 'Estimated', 'Countries', 'NCT Number'])

In [8]:
#Create csv file
enrollment_countries_respiratory.to_csv('enrollment_countries_respiratory.csv',index=False)

In [9]:
my_df = pd.read_csv('enrollment_countries_respiratory.csv')